## Tutorials

*   https://huggingface.co/learn/nlp-course/chapter5/3?fw=pt
*   https://huggingface.co/docs/datasets/v1.1.1/loading_datasets.html

*   https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/en/chapter5/section3.ipynb#scrollTo=JRn46d_dxNHP




In [ ]:
# To resolve CSV error in MS Excel, save files to 
# Preprocess data with Tab delimiter
from datasets import load_dataset
import csv
#train_df = pd.read_csv("COVID_Train_Set.csv", header=0, index_col=0, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, engine='python' )

#test_df = pd.read_csv("COVID_Test_Set.csv", header=0, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, engine='python' )

#data_files = {"train": train_df, "test": test_df}

covid_dataset = load_dataset("csv", data_files={'train': 'COVID_Train_Set.csv', 'test': 'COVID_Test_Set.csv'}, delimiter=",")

In [ ]:
#Shuffle and select the first 1000 samples
covid_sample = covid_dataset["train"].shuffle(seed=10).select(range(75))
covid_sample[:5]

NameError: ignored

In [ ]:
# Creates AttributeError: 'NoneType' object has no attribute 'lower'

def lowercase_sentence(example):
    return {"sentence": example["sentence"].lower()}


covid_dataset.map(lowercase_sentence)

In [ ]:
def filter_nones(x):
    return x["sentence"] is not None

In [ ]:
(lambda x: x * x)(3)

In [ ]:
(lambda base, height: 0.5 * base * height)(4, 8)

In [ ]:
covid_dataset = covid_dataset.filter(lambda x: x["sentence"] is not None)

In [ ]:
covid_dataset = covid_dataset.map(lowercase_sentence)
# Check that lowercasing worked
covid_dataset["train"]["sentence"][:3]

In [ ]:
def compute_sentence_length(example):
    return {"sentence_length": len(example["sentence"].split())}

In [ ]:
covid_dataset = covid_dataset.map(compute_sentence_length)
# Inspect the first training example
covid_dataset["train"][0]

In [ ]:
covid_dataset["train"].sort("sentence_length")[:3]

In [ ]:
covid_dataset = covid_dataset.filter(lambda x: x["sentence_length"] > 30)
print(covid_dataset.num_rows)

In [ ]:
import html

text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [ ]:
covid_dataset = covid_dataset.map(lambda x: {"sentence": html.unescape(x["sentence"])})

In [ ]:
new_covid_dataset = covid_dataset.map(
    lambda x: {"sentence": [html.unescape(o) for o in x["sentence"]]}, batched=True
)

NameError: ignored

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

ModuleNotFoundError: ignored

In [ ]:
%time tokenized_dataset = covid_dataset.map(tokenize_function, batched=True)

In [ ]:
slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)


def slow_tokenize_function(examples):
    return slow_tokenizer(examples["sentence"], truncation=True)


tokenized_dataset = covid_dataset.map(slow_tokenize_function, batched=True, num_proc=8)

NameError: ignored

In [ ]:
def tokenize_and_split(examples):
    return tokenizer(
        examples["sentence"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

In [ ]:
result = tokenize_and_split(covid_dataset["train"][0])
[len(inp) for inp in result["input_ids"]]

In [ ]:
# Causes error: ArrowInvalid: Column 6 named input_ids expected length 1000 but got length 1976
tokenized_dataset = covid_dataset.map(tokenize_and_split, batched=True)

NameError: ignored

In [ ]:
tokenized_dataset = covid_dataset.map(
    tokenize_and_split, batched=True, remove_columns=covid_dataset["train"].column_names
)

In [ ]:
len(tokenized_dataset["train"]), len(covid_dataset["train"])

In [ ]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples["sentence"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

In [ ]:
tokenized_dataset = covid_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset

In [ ]:
covid_dataset.set_format("pandas")

NameError: ignored

In [ ]:
covid_dataset["train"][:3]

In [ ]:
train_df = covid_dataset["train"][:]

In [ ]:
frequencies = (
    train_df["sentence"]
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "sentence", "sentence": "frequency"})
)
frequencies.head()

In [ ]:
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

In [ ]:
covid_dataset_clean = covid_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
covid_dataset_clean["validation"] = covid_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
covid_dataset_clean["test"] = covid_dataset["test"]
covid_dataset_clean

NameError: ignored

In [ ]:
covid_dataset_clean.save_to_disk("covid-reviews")

In [ ]:
from datasets import load_from_disk

covid_dataset_reloaded = load_from_disk("covid-reviews")
covid_dataset_reloaded

In [ ]:
for split, dataset in covid_dataset_clean.items():
    dataset.to_json(f"covid-reviews-{split}.jsonl")

In [ ]:
!head -n 1 covid-reviews-train.jsonl

head: cannot open 'covid-reviews-train.jsonl' for reading: No such file or directory


In [ ]:
data_files = {
    "train": "covid-reviews-train.jsonl",
    "validation": "covid-reviews-validation.jsonl",
    "test": "covid-reviews-test.jsonl",
}
covid_dataset_reloaded = load_dataset("json", data_files=data_files)

In [ ]:
# Setup the model
checkpoint = "bert-base-uncased"
# Use a CUDA device
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
# Setup the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
print(data_collator)

NameError: ignored

In [ ]:
# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    metric = load_metric("f1")
    return metric.compute(predictions=preds, references=labels)

In [ ]:
#training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "epoch",   # evaluation strategy to adopt during training
    learning_rate=2e-5,             # learning rate
    per_device_train_batch_size=3,  # batch size per device during training
    per_device_eval_batch_size=3,   # batch size for evaluation
    num_train_epochs=1,             # total number of training epochs
    weight_decay=0.01,              # strength of weight decay
    push_to_hub=False,
)
print(training_args)

NameError: ignored

In [ ]:
trainer.train()

In [ ]:
# Compute and print the F1 score
metric = load_metric("f1")
predictions = trainer.predict(tokenized_dataset["test"])
preds = predictions.predictions.argmax(-1)
f1_score = metric.compute(predictions=preds, references=predictions.label_ids)
print(f1_score)

NameError: ignored

In [ ]:
# Compute and print the F1 score
metric = load_metric("accuracy")
predictions = trainer.predict(tokenized_dataset["test"])
preds = predictions.predictions.argmax(-1)
accuracy_score = metric.compute(predictions=preds, references=predictions.label_ids)
print(accuracy_score)